In [1]:
%pip install tensorflow

Looking in indexes: https://pypi.yandex-team.ru/simple/
     |████████████████████████████████| 479.6 MB 59.2 MB/s eta 0:00:01    |██▉                             | 43.1 MB 35.6 MB/s eta 0:00:13     |█████▋                          | 84.5 MB 43.4 MB/s eta 0:00:10     |██████████▎                     | 154.1 MB 50.2 MB/s eta 0:00:07     |██████████████████▋             | 279.2 MB 47.4 MB/s eta 0:00:05     |█████████████████████           | 313.9 MB 34.1 MB/s eta 0:00:05     |████████████████████████████    | 420.5 MB 60.0 MB/s eta 0:00:01     |██████████████████████████████▊ | 461.3 MB 59.2 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 21.6 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 37.8 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 44.7 MB/s eta 0:00:01
     |████████████████████████████████| 440 kB 27.2 MB/s eta 0:00:01
     |████████████████████████████████| 5.4 MB 46.8 MB/s eta 0:00:01
     |████████████████████████████████| 17.3

In [1]:
%load_ext dotenv
%dotenv
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import boto3
# import tensorflow as tf
from warnings import filterwarnings
filterwarnings("ignore")
import os
os.environ["PYTHONWARNINGS"] = "ignore"

s3_client = boto3.client(
    's3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
)
BUCKET_NAME = 'tnn-hse-medtech'
DATASET_DIR = 'datasets/'

In [2]:
data = pd.read_csv('normalized_data.csv')

In [3]:
data

,expert,file_id,severity,relative_path
0,MedicalExpert-I,c7284c66fa8ec0ab4594e5dcd44866f408238f685e999e...,0,MedicalExpert-I/0Normal/NormalG0 (1).png
1,MedicalExpert-I,4a547b94fe02a7565beb21aa9195f4deffe831086da240...,0,MedicalExpert-I/0Normal/NormalG0 (10).png
2,MedicalExpert-I,ca8a296d1e15e0ed84c1ab82440426f28cd0582c19964f...,0,MedicalExpert-I/0Normal/NormalG0 (100).png
3,MedicalExpert-I,6f8bb6bbf0f4def4fdbe1a486064e47eefed0fa832246c...,0,MedicalExpert-I/0Normal/NormalG0 (101).png
4,MedicalExpert-I,d3c8f051ee6c5f59dff4657b7c007860be652e313d85ab...,0,MedicalExpert-I/0Normal/NormalG0 (102).png
...,...,...,...,...
3273,MedicalExpert-II,a762fc9fda75a15538789eb78b53a7ca12631e3c59993d...,4,MedicalExpert-II/4Severe/SevereG4 (95).png
3274,MedicalExpert-II,0ab9e6787dd93e3ad8505c6dcbafa955b91554b026c014...,4,MedicalExpert-II/4Severe/SevereG4 (96).png
3275,MedicalExpert-II,1b3dba34278b1ba238c7051be6886cab2cc2c1c985d368...,4,MedicalExpert-II/4Severe/SevereG4 (97).png
3276,MedicalExpert-II,3338d1a673dd92b49fc1967f0f3cea6bc645fea48bb84b...,4,MedicalExpert-II/4Severe/SevereG4 (98).png


In [5]:
s3_client.download_file?

Signature:
s3_client.download_file(
    Bucket,
    Key,
    Filename,
    ExtraArgs=None,
    Callback=None,
    Config=None,
)
Docstring:
Download an S3 object to a file.

Usage::

    import boto3
    s3 = boto3.resource('s3')
    s3.meta.client.download_file('mybucket', 'hello.txt', '/tmp/hello.txt')

Similar behavior as S3Transfer's download_file() method,
except that parameters are capitalized. Detailed examples can be found at
:ref:`S3Transfer's Usage <ref_s3transfer_usage>`.

:type Bucket: str
:param Bucket: The name of the bucket to download from.

:type Key: str
:param Key: The name of the key to download from.

:type Filename: str
:param Filename: The path to the file to download to.

:type ExtraArgs: dict
:param ExtraArgs: Extra arguments that may be passed to the
    client operation. For allowed download arguments see
    boto3.s3.transfer.S3Transfer.ALLOWED_DOWNLOAD_ARGS.

:type Callback: function
:param Callback: A method which takes a number of bytes transferred to
   

In [4]:
from pathlib import Path

from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
import PIL

IMAGE_DIRECTORY = Path('.') / 'images'

@dataclass()
class Metadata:
    severity: int
    relative_path: str
    path: Path
    width: int
    height: int
    size: int

def handle_file(item) -> Metadata:
    path = IMAGE_DIRECTORY / item.relative_path
    path.parent.mkdir(parents=True, exist_ok=True)
    s3_client.download_file(BUCKET_NAME, f'{DATASET_DIR}{item.relative_path}', path)
    with PIL.Image.open(str(path)) as im:
        width, height = im.size
    return Metadata(item.severity, item.relative_path, path, width, height, size=path.stat().st_size)

raw_data = []
with ThreadPoolExecutor(max_workers=100) as pool:
    for item in tqdm(pool.map(handle_file, data.itertuples(), chunksize=1), total=len(data)):
        raw_data.append(item)

  0%|          | 0/3278 [00:00<?, ?it/s]

In [69]:
df = pd.DataFrame(raw_data)
df

,severity,relative_path,path,width,height,size
0,0,MedicalExpert-I/0Normal/NormalG0 (1).png,images/MedicalExpert-I/0Normal/NormalG0 (1).png,300,162,31193
1,0,MedicalExpert-I/0Normal/NormalG0 (10).png,images/MedicalExpert-I/0Normal/NormalG0 (10).png,300,162,34614
2,0,MedicalExpert-I/0Normal/NormalG0 (100).png,images/MedicalExpert-I/0Normal/NormalG0 (100).png,300,162,33982
3,0,MedicalExpert-I/0Normal/NormalG0 (101).png,images/MedicalExpert-I/0Normal/NormalG0 (101).png,300,162,28115
4,0,MedicalExpert-I/0Normal/NormalG0 (102).png,images/MedicalExpert-I/0Normal/NormalG0 (102).png,300,162,31167
...,...,...,...,...,...,...
3273,4,MedicalExpert-II/4Severe/SevereG4 (95).png,images/MedicalExpert-II/4Severe/SevereG4 (95).png,300,162,28841
3274,4,MedicalExpert-II/4Severe/SevereG4 (96).png,images/MedicalExpert-II/4Severe/SevereG4 (96).png,300,162,32913
3275,4,MedicalExpert-II/4Severe/SevereG4 (97).png,images/MedicalExpert-II/4Severe/SevereG4 (97).png,300,162,34643
3276,4,MedicalExpert-II/4Severe/SevereG4 (98).png,images/MedicalExpert-II/4Severe/SevereG4 (98).png,300,162,30874


In [70]:
df.groupby(['width', 'height']).size()

width  height
300    162       2916
640    161        362
dtype: int64

In [71]:
double = df[df.width == 640]
single = df[df.width == 300]
single

,severity,relative_path,path,width,height,size
0,0,MedicalExpert-I/0Normal/NormalG0 (1).png,images/MedicalExpert-I/0Normal/NormalG0 (1).png,300,162,31193
1,0,MedicalExpert-I/0Normal/NormalG0 (10).png,images/MedicalExpert-I/0Normal/NormalG0 (10).png,300,162,34614
2,0,MedicalExpert-I/0Normal/NormalG0 (100).png,images/MedicalExpert-I/0Normal/NormalG0 (100).png,300,162,33982
3,0,MedicalExpert-I/0Normal/NormalG0 (101).png,images/MedicalExpert-I/0Normal/NormalG0 (101).png,300,162,28115
4,0,MedicalExpert-I/0Normal/NormalG0 (102).png,images/MedicalExpert-I/0Normal/NormalG0 (102).png,300,162,31167
...,...,...,...,...,...,...
3273,4,MedicalExpert-II/4Severe/SevereG4 (95).png,images/MedicalExpert-II/4Severe/SevereG4 (95).png,300,162,28841
3274,4,MedicalExpert-II/4Severe/SevereG4 (96).png,images/MedicalExpert-II/4Severe/SevereG4 (96).png,300,162,32913
3275,4,MedicalExpert-II/4Severe/SevereG4 (97).png,images/MedicalExpert-II/4Severe/SevereG4 (97).png,300,162,34643
3276,4,MedicalExpert-II/4Severe/SevereG4 (98).png,images/MedicalExpert-II/4Severe/SevereG4 (98).png,300,162,30874


In [81]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# classes = np.asarray(map(str, range(5))).reshape((-1, 1))

datagen = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
)
training_generator = datagen.flow_from_dataframe(
    dataframe=single, 
    directory=IMAGE_DIRECTORY,
    x_col='relative_path',
    y_col='severity',
    target_size=(300, 162),
    # classes=classes,
    # classes=['0', '1', '2', '3', '4'],
    # class_mode='categorical',
    interpolation='bicubic',
    subset='training',
    batch_size=128,
)
valid_generator = datagen.flow_from_dataframe(
    dataframe=single, 
    directory=IMAGE_DIRECTORY,
    x_col='relative_path',
    y_col='severity',
    target_size=(300, 162),
    # classes=classes,
    # classes=['0', '1', '2', '3', '4'],
    # class_mode='categorical',
    interpolation='bicubic',
    subset='validation',
    batch_size=128,
)

Found 2333 validated image filenames belonging to 5 classes.


Found 583 validated image filenames belonging to 5 classes.


In [82]:
from tensorflow.keras import layers, Model, Sequential

class ConvolutionalAutoEncoder(Model):
    def __init__(self):
        super().__init__()

        self.encoder = Sequential([
            layers.Input(shape=(300, 162, 3), name='e1'),
            # layers.Flatten(name='e2'),
            layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2, name='e3'),
            layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2, name='e4'),
        ], name='encoder')
        self.decoder = Sequential([
            layers.Conv2DTranspose(8, kernel_size=(3, 3), strides=2, activation='relu', padding='same', name='d1', output_padding=1),
            layers.Conv2DTranspose(16, kernel_size=(3, 3), strides=2, activation='relu', padding='same', name='d2', output_padding=1),
            # layers.GlobalAveragePooling2D(keepdims=True, name='d3'),
            layers.Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same', name='d4'),
        ], name='decoder')

    def call(self, x):
        latent_x = self.encoder(x)
        decoded_x = self.decoder(latent_x)
        return decoded_x

    
autoencoder = ConvolutionalAutoEncoder()
autoencoder.compile(optimizer='adam', loss='bce')
autoencoder.fit_generator(training_generator, epochs=1000)
autoencoder.encoder.summary()


Epoch 1/1000


ValueError: in user code:

    File "/home/tiunovnn/git/hse-knee-xray-research/models/ML/venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/tiunovnn/git/hse-knee-xray-research/models/ML/venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/tiunovnn/git/hse-knee-xray-research/models/ML/venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/tiunovnn/git/hse-knee-xray-research/models/ML/venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/tiunovnn/git/hse-knee-xray-research/models/ML/venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 11

In [70]:
autoencoder.decoder.build()

ValueError: You must provide an `input_shape` argument.

In [80]:
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
     |████████████████████████████████| 186.7 MB 7.5 kB/s  eta 0:00:01
     |████████████████████████████████| 1.6 MB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 5.7 MB 66.1 MB/s eta 0:00:01
     |████████████████████████████████| 153 kB 69.5 MB/s eta 0:00:01
     |████████████████████████████████| 536 kB 61.8 MB/s eta 0:00:01
ERROR: tensorflow 2.13.1 has requirement typing-extensions<4.6.0,>=3.6.6, but you'll have typing-extensions 4.8.0 which is incompatible.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.5.0
    Uninstalling typing-extensions-4.5.0:
      Successfully uninstalled typing-extensions-4.5.0
Note: you may need to restart the kernel to use updated packages.


In [16]:
%pip install torchsummary

Looking in indexes: https://pypi.yandex-team.ru/simple/
Note: you may need to restart the kernel to use updated packages.


In [76]:
import torch
from torch import nn, optim
from torchvision.datasets import ImageFolder
from torchvision import transforms

SINGLE_PATH = Path('.') / 'single'
# for item in single.itertuples():
#     directory = SINGLE_PATH / str(item.severity)
#     directory.mkdir(parents=True, exist_ok=True)
#     new_path =  directory / Path(item.path.name)
#     Path(item.path).rename(new_path)

train_transforms = transforms.Compose([#transforms.RandomRotation(30),
                                      #transforms.RandomResizedCrop(224),
                                      #transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor()])
dataset = ImageFolder(SINGLE_PATH, transform=train_transforms)
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=32)


In [8]:
dataiter = iter(dataset_loader)
images, labels = next(dataiter)
print(torch.min(images), torch.max(images))

tensor(0.) tensor(0.8275)


In [63]:
class TorchConvAutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1), # -> N, 16, 150, 81
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1), # -> N, 32, 75, 41
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1), # -> N, 64, 38, 21
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1), # -> N, 128, 19, 11
            nn.ReLU(),
            nn.Conv2d(128, 256, 3, stride=2, padding=1), # -> N, 256, 10, 6
            nn.ReLU(),
            nn.Conv2d(256, 512, (6, 10)), # -> N, 512, 1, 1
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, (6, 10)), # -> N, 256, 10, 6
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=0), # -> N, 128, 19, 11
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=[0, 1]), # -> N, 64, 38, 21
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=0), # -> N, 32, 75, 41
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=[0, 1]), # -> N, 16, 150, 81
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, 3, stride=2, padding=1, output_padding=1), # -> N, 1, 300, 162
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [64]:
model = TorchConvAutoEncoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

In [65]:
from torchsummary import summary

summary(model.encoder, (3, 162, 300))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 16, 81, 150]             448
              ReLU-2          [-1, 16, 81, 150]               0
            Conv2d-3           [-1, 32, 41, 75]           4,640
              ReLU-4           [-1, 32, 41, 75]               0
            Conv2d-5           [-1, 64, 21, 38]          18,496
              ReLU-6           [-1, 64, 21, 38]               0
            Conv2d-7          [-1, 128, 11, 19]          73,856
              ReLU-8          [-1, 128, 11, 19]               0
            Conv2d-9           [-1, 256, 6, 10]         295,168
             ReLU-10           [-1, 256, 6, 10]               0
           Conv2d-11            [-1, 512, 1, 1]       7,864,832
Total params: 8,257,440
Trainable params: 8,257,440
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.56
Forw

In [66]:
summary(model.decoder, (512, 1, 1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1           [-1, 256, 6, 10]       7,864,576
              ReLU-2           [-1, 256, 6, 10]               0
   ConvTranspose2d-3          [-1, 128, 11, 19]         295,040
              ReLU-4          [-1, 128, 11, 19]               0
   ConvTranspose2d-5           [-1, 64, 21, 38]          73,792
              ReLU-6           [-1, 64, 21, 38]               0
   ConvTranspose2d-7           [-1, 32, 41, 75]          18,464
              ReLU-8           [-1, 32, 41, 75]               0
   ConvTranspose2d-9          [-1, 16, 81, 150]           4,624
             ReLU-10          [-1, 16, 81, 150]               0
  ConvTranspose2d-11          [-1, 3, 162, 300]             435
Total params: 8,256,931
Trainable params: 8,256,931
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forw

In [78]:
from tqdm.notebook import tqdm

num_epochs = 10
outputs = []
with tqdm(total=num_epochs*len(dataset_loader)) as pbar:
    for epoch in range(num_epochs):
        for img, _ in dataset_loader:
            recon = model(img)
            loss = criterion(recon, img)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pbar.update(1)

    print(f'Epoch: {epoch + 1}, Loss: {loss.item():.4f}')
    outputs.append((epoch, img, recon))

  0%|          | 0/460 [00:00<?, ?it/s]